In [7]:
import sys
import subprocess

def install_packages():
    packages = ["requests", "pandas"]
    for package in packages:
        try:
            __import__(package)
        except ImportError:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install_packages()

import requests
import pandas as pd

def summarize_with_serpapi(query, api_key):
    url = "https://serpapi.com/search"
    params = {
        "engine": "google",
        "q": query,
        "api_key": api_key
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.RequestException as e:
        print(f"Request failed for query '{query}': {e}")
        return "Summary fetch error"

    summary = data.get("answer_box", {}).get("snippet") or \
              data.get("organic_results", [{}])[0].get("snippet") or \
              data.get("knowledge_graph", {}).get("description") or \
              "No Summary Available"

    return summary

def process_csv(file_path, api_key):
    df = pd.read_csv(file_path)
    if "Title" not in df.columns:
        print("No 'Title' column found.")
        return

    df.drop_duplicates(subset=["Title"], inplace=True)  # Deduplication
    df["Title"] = df["Title"].astype(str).str.strip()  # Enrichment/cleanup

    df["Summary"] = df["Title"].apply(lambda x: summarize_with_serpapi(x, api_key) if pd.notna(x) else "No Summary")
    df.dropna(subset=["Summary"], inplace=True)  # Validation
    df = df[df["Summary"] != "No Summary Available"]  # Remove generic fallback

    output_file = "summarized_data.csv"
    df.to_csv(output_file, index=False)
    print(f"Summarized results saved to {output_file}")

if __name__ == "__main__":
    csv_file = input("Enter CSV file path: ")
    api_key = input("Enter SerpAPI key: ")
    process_csv(csv_file, api_key)

Enter CSV file path: /content/google_search_results.csv
Enter SerpAPI key: your-serpapi-key-here
Summarized results saved to summarized_data.csv
